In [1]:
import firebase_admin
from firebase_admin import credentials, db, storage
from math import floor, sqrt
import ast
import smtplib
import plotly.express as px
import pandas as pd
import json
import streamlit as st
import plotly.graph_objects as go

In [2]:
faction_list = [
    "🧝 Unaffilated",
    "🏴 Blackthorne Company",
    "💰 Guild of the Black Sky",
    "🛡 Eponore",
    "⚜️ Catalpa",
    "🍷 Cedar Hill",
    "🧛‍♂️ The Dismissed",
    "💀 Geth",
    "❄️ Grimfrost",
    "🌳 The Grove",
    "🌙 The Irregulars",
    "⚖️ The Order",
    "🎪 Prismatic Troupe",
    "⚔️ Sunsteel Company",
    "🦁 Kult of Tharros",
    "🐴 Vidarian Khanate",
    "🏹 The Wardens",
    "🕊️ The White Ravens "
]

path_list = [
    '🗡 Warrior',
    '🪤 Rogue',
    '🩹 Healer',
    '🔮 Wizard'
]

def get_tier(events):
    return floor((sqrt(8*events)-1)/2)

if not firebase_admin._apps:
    key_dict = json.loads(st.secrets["firebase"], strict=False)
    creds = credentials.Certificate(key_dict)
    defualt_app = firebase_admin.initialize_app(creds, {
        'databaseURL': 'https://la-character-sheets-default-rtdb.firebaseio.com',
        'storageBucket':'la-character-sheets.appspot.com'
    })

In [3]:
user_data = db.reference("users/").get()
user_table = []
for key in user_data.keys():
    try:
        user_events = pd.DataFrame(json.loads(user_data[key]['event_info']))
        user_events.reset_index(drop=True, inplace=True)
        tier = get_tier(len(user_events[user_events['Event Type'] != "🪚 Work Weekend"]))
        skill_points = int(user_events["Skill Points"].sum())
        try:
            avail_points = int(user_events["Skill Points"].sum()) - int(user_data[key]['point_spend'])
        except:
            avail_points = skill_points
    except:
        skill_points = 0
        tier = 0
        avail_points = skill_points

    user_table.append({
        'Username':key,
        'Player':user_data[key]['name'],
        'Character':user_data[key]['character_name'],
        'Faction':user_data[key]['faction'],
        'Path':user_data[key]['path'],
        'Tier':tier,
        'Earned Points':skill_points,
        "Available Points":avail_points
    })
user_df = pd.DataFrame(user_table)

KeyError: 'name'

In [ ]:
user_df

,Username,Player,Character,Faction,Path,Tier,Earned Points,Available Points
0,caenis_macneary,Niki Hunter,Caenis MacNeary,🎪 Prismatic Troupe,🗡 Warrior,4,28,0
1,ghostfox,Kara,Elora,🎪 Prismatic Troupe,🕳 Rogue,0,1,1
2,ntdens,Nate Densmore,Kython,🎪 Prismatic Troupe,🩸 Healer,5,43,11


In [ ]:
tier_df = user_df.groupby('Tier').count()['Username']
tier_list = pd.DataFrame({'Tier':list(range(0,11))})
tier_df = tier_list.merge(tier_df, how='left', on='Tier').fillna(0).rename(columns={'Username':'Players'})
go.Figure(go.Bar(x=tier_df['Tier'], y=tier_df['Players']))

In [ ]:
player_events = []
for player in user_df['Username']:
    try:
        user_events = pd.DataFrame(json.loads(user_data[player]['event_info']))
        user_events.reset_index(drop=True, inplace=True)
        user_events = user_events[user_events['Event Type'] != "🪚 Work Weekend"]
        try:
            user_events['Event Date'] = pd.to_datetime(user_events['Event Date'], format="%B %Y")
        except:
            pass
        try:
            user_events['Event Date'] = pd.to_datetime(user_events['Event Date'], unit='ms')
        except:
            pass
        player_events.append(pd.DataFrame({'Date':list(user_events['Event Date']),'Player':player}))
    except:
        pass

In [ ]:
attend = pd.concat(player_events)
attend['Date'] = attend.Date - pd.offsets.MonthEnd(0) - pd.offsets.MonthBegin(1)
attend = attend.groupby('Date').count().reset_index()
px.line(attend, x='Date', y='Player', title='Attendance Over Time')

In [ ]:
bucket = storage.bucket()

In [ ]:
blob = bucket.blob('ntdens')

In [ ]:
import PIL.Image as Image
import io

In [ ]:
image_location = user_data['ntdens']['pic_name']

In [ ]:
image_location

'ntdens/profile_pic.png'

In [ ]:
for b in bucket.list_blobs(prefix='ntdens'):
    if b.name != image_location:
        b.delete()

In [ ]:
(st.secrets['admins'] + list(st.secrets['faction_leaders']))

['ntdens', 'caenis_macneary']

In [ ]:
key = 'ntdens'
user_data = db.reference("users/").get()
user_auth = db.reference("auth").child('credentials/usernames/').get()
tab1, tab2,tab3 = st.tabs(['Player List', 'Character View', 'Event View'])
user_table = []
for key in user_data.keys():
    try:
        user_events = pd.DataFrame(json.loads(user_data[key]['event_info']))
        user_events.reset_index(drop=True, inplace=True)
        tier = get_tier(len(user_events[user_events['Event Type'] != "🪚 Work Weekend"]))
        skill_points = int(user_events["Skill Points"].sum())
        try:
            avail_points = int(user_events["Skill Points"].sum()) - int(user_data[key]['point_spend'])
        except:
            avail_points = skill_points
    except:
        skill_points = 0
        tier = 0
        avail_points = skill_points
    user_table.append({
        'Username':key,
        'Player':user_auth[key]['name'],
        'Character':user_data[key]['character_name'],
        'Faction':user_data[key]['faction'],
        'Path':user_data[key]['path'],
        'Tier':tier,
        'Earned Points':skill_points,
        "Available Points":avail_points
    })

In [ ]:
user_df = pd.DataFrame(user_table)

In [ ]:
character_data = user_data[key]
player_data = pd.DataFrame({
    'Category': ['Character: ','Player: ','Path: ','Faction: ','Tier: ','Skill Points: '],
    'Information': [character_data['character_name'],character_data['name'],character_data['path'],character_data['faction'],user_df[user_df['Username'] == key]['Tier'].values[0],user_df[user_df['Username'] == key]['Available Points'].values[0]]
                    })

In [ ]:
import emoji
import requests
from PIL import Image as ImageCheck
from unicodedata import normalize
def replace_with_emoji_pdf(text, size):
    """
    Reportlab's Paragraph doesn't accept normal html <image> tag's attributes
    like 'class', 'alt'. Its a little hack to remove those attrbs
    """

    for e in emoji.analyze(text):
        e_icon = e.chars
        try:
            emoji_code = "-".join(f"{ord(c):x}" for c in e_icon)
            url = f"https://raw.githubusercontent.com/twitter/twemoji/master/assets/72x72/{emoji_code}.png"
            im = ImageCheck.open(requests.get(url, stream=True).raw)
            text = text.replace(e_icon, '<img height={} width={} src="{}"/>'.format(size, size, url))
        except:
            emoji_code = [f"{ord(c):x}" for c in e_icon][0]
            url = f"https://raw.githubusercontent.com/twitter/twemoji/master/assets/72x72/{emoji_code}.png"
            im = ImageCheck.open(requests.get(url, stream=True).raw)
            text = text.replace(e_icon, '<img height={} width={} src="{}"/>'.format(size, size, url))
    return normalize('NFKD', text).encode('ascii','ignore')

In [ ]:
df = pd.read_excel('Skills_Table.xlsx')
known = user_data['ntdens']['known']
known = ast.literal_eval(known)
known_data = df[df['Skill Name'].isin(known)]
display_data = known_data[['Skill Name', 'Limitations', 'Phys Rep']].drop_duplicates(subset=['Skill Name']).copy()
display_data = display_data.fillna('')

In [ ]:
df[df.Tier.between(0,0)]

,Skill Name,Description,Path,Tier,Limitations,Phys Rep,Prerequisite,Spell
0,Basic Weapon Proficiency,Players learn the basic combat and safety rule...,Warrior,0,NaN,NaN,NaN,False
1,Armor Proficiency,Players learn about the armor that they wear a...,Warrior,0,NaN,NaN,NaN,False
2,Kindle Flame/Torch,Player gains proficiency at creating normal fi...,Warrior,0,10’ radius,Optional (larp safe electronically lighted tor...,NaN,False
3,Shield Control,Players learn the art of Shield Control and ma...,Warrior,0,NaN,NaN,NaN,False
29,Loot,Player gains the ability to loot bodies left o...,Rogue,0,Players cannot [Loot] another player's persona...,Player holds hand over the location being loot...,NaN,False
30,Detect Trap,"Using this ability, Rogues can now see traps l...",Rogue,0,Each search is limited to a single item or are...,"Role play, 15 seconds. The player detecting tr...",NaN,False
31,Shield Training,Rogues gain the ability to safely use small to...,Rogue,0,NaN,NaN,NaN,False
32,Basic Weapon Proficiency,Players learn the basic combat and safety rule...,Rogue,0,NaN,NaN,NaN,False
33,Armor Proficiency,Players learn about the armor that they may we...,Rogue,0,NaN,NaN,NaN,False
34,Kindle Flame/Torch,Player gains proficiency at creating normal fi...,Rogue,0,10’ radius,Optional (larp safe electronically lighted tor...,NaN,False


In [ ]:
user_events = pd.DataFrame(json.loads(user_data['ntdens']['event_info']))
user_events.reset_index(drop=True, inplace=True)
try:
    user_events['Event Date'] = pd.to_datetime(user_events['Event Date'], format="%B %Y").apply(lambda x:x.strftime("%B %Y"))
except:
    pass
try:
    user_events['Event Date'] = pd.to_datetime(user_events['Event Date'], unit='ms').apply(lambda x:x.strftime("%B %Y"))
except:
    pass
user_events[['Bonus Skill Points', 'Skill Points']] = user_events[['Bonus Skill Points', 'Skill Points']].astype(int)

In [ ]:
from reportlab.platypus import *
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter, portrait
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import numpy as np
from reportlab.lib.enums import TA_LEFT, TA_CENTER
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
PAGE_WIDTH, PAGE_HEIGHT= letter
styles = getSampleStyleSheet()

PAGESIZE = portrait(letter)

font_file = 'SedanSC-Regular.ttf'
sedan_font = TTFont('SedanSC', font_file)
pdfmetrics.registerFont(sedan_font)


Title = "LARP Adventures Character Sheet"
pageinfo = "platypus example"
def myFirstPage(canvas, doc):
    canvas.saveState()
    canvas.drawImage('OLD_PAPER_TEXTURE.jpg',0,0)
    canvas.drawImage('la_logo.png', doc.leftMargin, doc.height + doc.bottomMargin + doc.topMargin - 4*cm, 3*cm, 3*cm, mask='auto')
    canvas.setFont('SedanSC',16)
    canvas.drawCentredString(PAGE_WIDTH/2.0, PAGE_HEIGHT-doc.topMargin, Title)
    canvas.setFont('SedanSC',9)
    canvas.drawString(inch, 0.75 * inch, "Page %d" % (doc.page))
    canvas.restoreState()

def myLaterPages(canvas, doc):
    canvas.saveState()
    canvas.drawImage('OLD_PAPER_TEXTURE.jpg',0,0)
    canvas.setFont('SedanSC',9)
    canvas.drawString(inch, 0.75 * inch, "Page %d" % (doc.page))
    canvas.restoreState()



character_info_style = TableStyle(
    [
        ('INNERGRID', (0,0), (-1,-1), 0.25, colors.black),
    ]
)

skill_info_style = TableStyle(
    [
        ('INNERGRID', (0,0), (-1,-1), 0.25, colors.black),
        ('BOX', (0,0), (-1,-1), 0.25, colors.black),
    ]
)


styles["Title"].fontName = 'SedanSC'
styles["Title"].fontSize = 10
styles["Title"].alignment = TA_LEFT

break_style = ParagraphStyle('breakstyle',
    fontSize=14,
    fontName='SedanSC',
    alignment = TA_CENTER
)

def table_gen(table_data, headers=False, tstyle=character_info_style):
    table_data = table_data.map(lambda x:replace_with_emoji_pdf(x, styles['Title'].fontSize) if isinstance(x, str) else str(x))
    if headers:
        t1 = Table([[Paragraph(col, style=styles['Title']) for col in table_data.columns]] + np.array(table_data.map(lambda x:Paragraph(x, style=styles['Title']))).tolist(), style=tstyle, repeatRows=1)
    else:
        t1 = Table(np.array(table_data.map(lambda x:Paragraph(x, style=styles['Title']))).tolist(), style=tstyle, repeatRows=1)
    return t1
t1 = table_gen(player_data)
doc = SimpleDocTemplate("table.pdf", pagesize=letter)

table_style = TableStyle(
    [
        ('INNERGRID', (0,0), (-1,-1), 0.25, colors.black),
        ('BOX', (0,0), (-1,-1), 0.25, colors.black),
        ('SPAN', (0, 0), (0, -1)),
        ('ALIGN', (0,0), (-1,-1), 'CENTER'),
        ('VALIGN', (0,0), (-1,-1), 'CENTER')
    ]
)
profile = Image("Kython.jpg",width=4*inch,height=4*inch,kind='proportional')
logo = Image('🎪 Prismatic Troupe.jpg',width=4*inch,height=2*inch,kind='proportional')

data_table = [
    [profile, t1], 
    ['', logo],
]
final_table = Table(data_table, style=table_style)

t2 = table_gen(display_data, headers=True, tstyle=skill_info_style)

t3 = table_gen(user_events, headers=True, tstyle=skill_info_style)

doc = SimpleDocTemplate("character_sheet.pdf", pagesize=PAGESIZE)
Story = [Spacer(1,1*inch)]
style = styles["Normal"]
Story.append(final_table)
Story.append(Spacer(1,1*inch))
Story.append(Paragraph('<u>Skills</u>', style=break_style))
Story.append(Spacer(1,.5*inch))
Story.append(t2)
Story.append(Spacer(1,1*inch))
Story.append(Paragraph('<u>Events</u>', style=break_style))
Story.append(Spacer(1,.5*inch))
Story.append(t3)
doc.build(Story, onFirstPage=myFirstPage, onLaterPages=myLaterPages)

In [ ]:
faction = "🎪 Prismatic Troupe"
user_data = db.reference("users/").child('ntdens').get()
image_location = user_data['pic_name']
bucket = storage.bucket()
blob = bucket.blob(image_location)
blob.download_to_filename(user_data['pic_name'].split('/')[1])
profile_image = user_data['pic_name'].split('/')[1]
if faction != "🧝 Unaffilated" or "🤖 NPC":
    blob = bucket.blob("faction_logos/{}.jpg".format(faction))
    blob.download_to_filename(faction + '.jpg')
    logo_image = faction + '.jpg'
else:
    blob = bucket.blob("faction_logos/la_logo.jpg".format(faction))
    blob.download_to_filename('la_logo.jpg')
    logo_image = 'la_logo.jpg'


In [5]:
import re

In [6]:
ky = "Kython"
test = re.compile(fr"^{ky}\.[a-zA-Z]{{3,4}}$")

In [8]:
if test.match("Kython.jpg"):
    print('hello')

hello


In [ ]:
import streamlit_authenticator as stauth

RuntimeError: Runtime hasn't been created!